# 제조보고서 원단 '타입/반품', '사용' 내역찾기

# 프로그램 개요

- 아래 프로그램은 제조보고서 2022년 04월 'FQMADU' 아이템의 기말재고가 -0.5 야드가 나와서 원인규명을 위해 제작했다.
- 제조보고서 아이템(완제품)중 바지나 치마같은 제품은 물량이 많아 문제가 생겼을 때 원단 추적이 까다롭다.
- 타입/반품과 사용은 + - 관계
- 타입 총량과 사용 총량을 sum 하여 튀어나오는 order를 잡아낸다.
- 튀어나온 오더로 재검색하고 입출고 내역을 확인하면 문제가 생긴 시점을 알 수 있다.
    - 원래 쿼리에서 group by로 sum한 쿼리를 개별건으로 해체해놔서 검색하면 모든 개별 order 내역이 나온다.
- 덤으로 회수안된 샘플오더가 로직상 같이 걸려나온다.

In [1]:
from sqlalchemy import create_engine
import pandas as pd
import binascii   # 한글 변환에 필요한 라이브러리
import sys
import pyautogui
from datetime import datetime

### DB 접속

In [2]:
sys.path.append('/settings')
import config

def connect_db(sid):
    if sid != config.COMPANY_DB_CONFIG['sid']:
        raise ValueError("DB 를 찾을 수 없습니다.")
    
    conn = create_engine(
        "oracle+cx_oracle://{user}:{password}@{host}:{port}/{sid}?encoding=UTF-8&nencoding=UTF-8".format(
            user=config.COMPANY_DB_CONFIG['user'],
            password=config.COMPANY_DB_CONFIG['password'],
            host=config.COMPANY_DB_CONFIG['host'],
            port=config.COMPANY_DB_CONFIG['port'],
            sid=config.COMPANY_DB_CONFIG['sid']
        )
    )
    
    return conn

engine = connect_db('IVY')

In [3]:
item = pyautogui.prompt(title='아이템',
                        text='제조보고서의 확인할 아이템(완제품 품번)을 입력하세요.\n여러개 입력시 , 로 구분하세요.\n예 : FQMADU, FJFBDU, FHMBAA',
                        default='FQMADU')
item

'FQMADU'

In [4]:
item = str(item.strip().replace(' ', '').upper().split(','))[1:-1]
item

"'FQMADU'"

#### 오늘 날짜

In [5]:
this_year = datetime.today().strftime('%Y')
this_month = datetime.today().strftime('%m')
last_year = str(int(this_year)-1)

this_year, this_month, last_year

('2022', '06', '2021')

### 시작월

In [6]:
start_month = pyautogui.prompt(title='조회기간 시작 월',
                              text=f'조회기간 시작 월을 입력하세요.\n예 : {last_year+"06"}',
                              default=f'{last_year+"06"}')
start_month

'202106'

### 종료월

In [7]:
end_month = pyautogui.prompt(title='조회기간 종료 월',
                             text=f'조회기간 종료 월을 입력하세요.\n예 : {this_year + this_month}',
                             default=f'{this_year + this_month}')
end_month

'202204'

### SQL 본문

#### 타입/반품 쿼리

- i_chulgo_t 테이블
    - cdate 기준으로 정산
    - 기간 : 전년 6월 ~ 문제생긴 달
    - qty 가 양수이면 타입, 음수면 반품

In [8]:
sql1 = f'''
SELECT '1'                              gubun,
       master_order                     order_no,
       master_status                    st,
       Rawtohex(utl_raw.Cast_to_raw(sch_name)) sch_name,
       chulgo_item                      item,
       stock_paitem                     paitem,
       chulgo_sojae                     sojae,
       chulgo_millon                    millon,
       chulgo_cdate                     cdate,
       Decode(chulgo_fact, 'VPF22', 'VPF31',
                           chulgo_fact) fact,
       chulgo_qty                       qty
FROM   i_chulgo_t,
       i_stock_t,
       i_suju_master_t,
       i_sch_t
WHERE  chulgo_cdate BETWEEN To_date('{start_month[:4]}'
                                    ||'{start_month[-2:]}'
                                    ||'01 000000', 'yyyymmdd hh24:mi:ss') AND
                            Last_day(
                                                        To_date('{end_month[:4]}'
                                                                ||'{end_month[-2:]}'
                                                                ||'01 235959',
                                                        'yyyymmdd hh24:mi:ss'))
       AND chulgo_item IN ({item})
       AND chulgo_millon = stock_millon
       AND chulgo_order = master_order
       AND ( master_remake = 'H'
              OR master_agent = 'VZ001' )
       AND sch_code(+) = master_school
UNION ALL
SELECT '2'                              gubun,
       master2_order                    order_no,
       master2_status                   st,
       Rawtohex(utl_raw.Cast_to_raw(sch_name)) sch_name,
       chulgo_item                      item,
       stock_paitem                     paitem,
       chulgo_sojae                     sojae,
       chulgo_millon                    millon,
       chulgo_cdate                     cdate,
       Decode(chulgo_fact, 'VPF22', 'VPF31',
                           chulgo_fact) fact,
       chulgo_qty                       qty
FROM   i_chulgo_t,
       i_stock_t,
       i_suju_master2_t,
       i_sch_t
WHERE  chulgo_cdate BETWEEN To_date('{start_month[:4]}'
                                    ||'{start_month[-2:]}'
                                    ||'01 000000', 'yyyymmdd hh24:mi:ss') AND
                            Last_day(
                                                        To_date('{end_month[:4]}'
                                                                ||'{end_month[-2:]}'
                                                                ||'01 235959',
                                                        'yyyymmdd hh24:mi:ss'))
       AND chulgo_item IN ({item})
       AND chulgo_millon = stock_millon
       AND chulgo_order = master2_order
       AND master2_remake = 'S'
       AND master2_jaepum <> 'P'
       AND master2_squota IS NOT NULL
       AND sch_code(+) = master2_school
UNION ALL
SELECT '3'                              gubun,
       master_order                     order_no,
       master_status                    st,
       Rawtohex(utl_raw.Cast_to_raw(sch_name)) sch_name,
       chulgo_item                      item,
       stock_paitem                     paitem,
       chulgo_sojae                     sojae,
       chulgo_millon                    millon,
       chulgo_cdate                     cdate,
       Decode(chulgo_fact, 'VPF22', 'VPF31',
                           chulgo_fact) fact,
       chulgo_qty                       qty
FROM   i_chulgo_t,
       i_stock_t,
       i_suju_master_t,
       i_sch_t
WHERE  chulgo_cdate BETWEEN To_date('{start_month[:4]}'
                                    ||'{start_month[-2:]}'
                                    ||'01 000000', 'yyyymmdd hh24:mi:ss') AND
                            Last_day(
                                                        To_date('{end_month[:4]}'
                                                                ||'{end_month[-2:]}'
                                                                ||'01 235959',
                                                        'yyyymmdd hh24:mi:ss'))
       AND chulgo_item IN ({item})
       AND chulgo_millon = stock_millon
       AND chulgo_order = master_order
       AND master_remake <> 'H'
       AND sch_code(+) = master_school 
'''

#### 사용량 쿼리

- 정보가 타입쪽에 비해 모자람.
- 밀넘버, 소재코드, 봉제업체가 없다.

In [9]:
sql2 = f'''
SELECT '1'                   gubun,
       master_order          order_no,
       master_status         st,
       Rawtohex(utl_raw.Cast_to_raw(sch_name)) sch_name,
       master_item           item,
       prods_paitem          paitem,
       prods_cdate           cdate,
       Nvl(prods_use_qty, 0) qty
FROM   i_suju_fact_t,
       i_prod_sub_t,
       i_suju_master_t,
       i_sch_t
WHERE  prods_cdate BETWEEN To_date('{start_month[:4]}'
                                   ||'{start_month[-2:]}'
                                   ||'01 000000', 'yyyymmdd hh24:mi:ss') AND
                           Last_day(
                                                      To_date('{end_month[:4]}'
                                                              ||'{end_month[-2:]}'
                                                              ||'01 235959',
                                                      'yyyymmdd hh24:mi:ss'))
       AND master_item IN ({item})
       AND prods_order = fact_order
       AND fact_order = master_order
       AND ( master_remake = 'H'
              OR master_agent = 'VZ001' )
       AND sch_code(+) = master_school
UNION ALL
SELECT '2'                   gubun,
       master2_order         order_no,
       master2_status        st,
       Rawtohex(utl_raw.Cast_to_raw(sch_name)) sch_name,
       master2_item          item,
       prods_paitem          paitem,
       prods_cdate           cdate,
       Nvl(prods_use_qty, 0) qty
FROM   i_suju_fact_t,
       i_prod_sub_t,
       i_suju_master2_t,
       i_sch_t
WHERE  prods_cdate BETWEEN To_date('{start_month[:4]}'
                                   ||'{start_month[-2:]}'
                                   ||'01 000000', 'yyyymmdd hh24:mi:ss') AND
                           Last_day(
                                                      To_date('{end_month[:4]}'
                                                              ||'{end_month[-2:]}'
                                                              ||'01 235959',
                                                      'yyyymmdd hh24:mi:ss'))
       AND master2_item IN ({item})
       AND prods_order = fact_order
       AND fact_order = master2_order
       AND master2_remake = 'S'
       AND master2_jaepum <> 'P'
       AND master2_squota IS NOT NULL
       AND sch_code(+) = master2_school
UNION ALL
SELECT '3'                   gubun,
       master_order          order_no,
       master_status         st,
       Rawtohex(utl_raw.Cast_to_raw(sch_name)) sch_name,
       master_item           item,
       prods_paitem          paitem,
       prods_cdate           cdate,
       Nvl(prods_use_qty, 0) qty
FROM   i_suju_fact_t,
       i_prod_sub_t,
       i_suju_master_t,
       i_sch_t
WHERE  prods_cdate BETWEEN To_date('{start_month[:4]}'
                                   ||'{start_month[-2:]}'
                                   ||'01 000000', 'yyyymmdd hh24:mi:ss') AND
                           Last_day(
                                                      To_date('{end_month[:4]}'
                                                              ||'{end_month[-2:]}'
                                                              ||'01 235959',
                                                      'yyyymmdd hh24:mi:ss'))
       AND master_item IN ({item})
       AND prods_order = fact_order
       AND fact_order = master_order
       AND master_remake <> 'H'
       AND master_agent <> 'VZ001'
       AND sch_code(+) = master_school
'''

### 한글변환 함수

In [10]:
# # US7ASCII -> CP949(완성형한글) 로 변환
def us7ascii_to_cp949(df):
    for index, byte_data in enumerate(df):
        if byte_data == None: # null 값이면 패스. 안하면 변환 에러난다.
            continue
        byte_data = binascii.unhexlify(df[index])  # 16진수 문자열 hexstr로 표현된 바이너리 데이터를 반환. 역함수는 b2a_hex()
        df[index] = byte_data.decode("cp949")  # 바이트 변환값 -> cp949(완성형 한글) 로 변환
    return df

### 표준정보 쿼리 함수

In [11]:
def sql_query(oracleSql):
    df = pd.read_sql_query(oracleSql, engine)

    if 'tkyk_name' in df.columns: # 해당컬럼이 없어도 에러없이 처리
        df_temp = df['tkyk_name'].copy()
        df['tkyk_name'] = us7ascii_to_cp949(df_temp)

    if 'agen_name' in df.columns:
        df_temp = df['agen_name'].copy()
        df['agen_name'] = us7ascii_to_cp949(df_temp)

    if 'sch_name' in df.columns:
        df_temp = df['sch_name'].copy()
        df['sch_name'] = us7ascii_to_cp949(df_temp)
    
    return df

In [12]:
df_taip = sql_query(sql1)
df_ta = df_taip[['order_no','st','sch_name','item','paitem','sojae','millon','cdate','fact','qty']].copy()
df_ta

,order_no,st,sch_name,item,paitem,sojae,millon,cdate,fact,qty
0,22NBC014Q-1,60,춘천남고,FQMADU,DU,UGD0058,19FGS030507,2021-06-07,VTF09,2.0
1,22NRC224Q-1,60,범어고남,FQMADU,DU,UGD0421,20FGS030037,2021-06-07,VTF09,4.0
2,22NAC719Q-1,60,군포중앙고,FQMADU,DU,UGD0071,20FGS030254,2021-06-07,VTF09,2.0
3,22NAC336Q-1,60,광주중앙고,FQMADU,DU,UGD0018,21FGS030004,2021-06-07,VTF09,2.0
4,22NSC399Q-1,60,서울국제고남,FQMADU,DU,UGD0018,21FGS030004,2021-06-07,VTF09,4.0
...,...,...,...,...,...,...,...,...,...,...
1521,22NDC115Q 37,60,공통회색바지(USX038 T 7),FQMADU,DU,USX0038,22FGS030021,2022-03-03,VTF09,45.0
1522,22NDC097Q 36,60,공통회색바지(USX038 S 7),FQMADU,DU,USX0038,22FGS030021,2022-03-03,VTF09,22.0
1523,22NDC076Q 37,60,공통곤색바지(UGD407 S 7),FQMADU,DU,UGD0407,22FGS030187,2022-03-03,VTF09,26.0
1524,22NSC151Q 4,60,중앙고남,FQMADU,DU,UGD0018,22FGS030421,2022-03-03,VTF09,54.0


In [13]:
df_sayoung = sql_query(sql2)
df_sayoung['millon'] = '' # 프레임 합치기를 위해 빈 column 생성
df_sayoung['fact'] = ''
df_sayoung['sojae'] = ''
df_sa = df_sayoung[['order_no','st','sch_name','item','paitem','sojae','millon','cdate','fact','qty']].copy()
df_sa['qty'] = df_sa['qty'] * (-1) # sum해서 0으로 만들거라 -1을 곱해준다.
df_sa

,order_no,st,sch_name,item,paitem,sojae,millon,cdate,fact,qty
0,22NAA022Q-1,60,성주중남,FQMADU,DU,,,2021-07-20,,-2.0
1,22NAC128Q-1,60,강화고,FQMADU,DU,,,2021-07-20,,-2.0
2,22NAC155Q-1,60,덕신고,FQMADU,DU,,,2021-07-20,,-6.0
3,22NAC280Q-1,60,효양고,FQMADU,DU,,,2021-07-20,,-2.0
4,22NAC330Q-1,60,공통곤색바지(시흥),FQMADU,DU,,,2021-07-20,,-2.0
...,...,...,...,...,...,...,...,...,...,...
1317,22NDC097Q 36,60,공통회색바지(USX038 S 7),FQMADU,DU,,,2022-03-10,,-22.0
1318,22NLC130Q 1,60,세인고,FQMADU,DU,,,2022-03-10,,-105.0
1319,22NSC151Q 4,60,중앙고남,FQMADU,DU,,,2022-03-10,,-54.0
1320,22NAC695Q 3,60,공통베이지바지(여주),FQMADU,DU,,,2022-03-11,,-41.0


In [14]:
df_calc = pd.concat([df_ta, df_sa])
df_calc

,order_no,st,sch_name,item,paitem,sojae,millon,cdate,fact,qty
0,22NBC014Q-1,60,춘천남고,FQMADU,DU,UGD0058,19FGS030507,2021-06-07,VTF09,2.0
1,22NRC224Q-1,60,범어고남,FQMADU,DU,UGD0421,20FGS030037,2021-06-07,VTF09,4.0
2,22NAC719Q-1,60,군포중앙고,FQMADU,DU,UGD0071,20FGS030254,2021-06-07,VTF09,2.0
3,22NAC336Q-1,60,광주중앙고,FQMADU,DU,UGD0018,21FGS030004,2021-06-07,VTF09,2.0
4,22NSC399Q-1,60,서울국제고남,FQMADU,DU,UGD0018,21FGS030004,2021-06-07,VTF09,4.0
...,...,...,...,...,...,...,...,...,...,...
1317,22NDC097Q 36,60,공통회색바지(USX038 S 7),FQMADU,DU,,,2022-03-10,,-22.0
1318,22NLC130Q 1,60,세인고,FQMADU,DU,,,2022-03-10,,-105.0
1319,22NSC151Q 4,60,중앙고남,FQMADU,DU,,,2022-03-10,,-54.0
1320,22NAC695Q 3,60,공통베이지바지(여주),FQMADU,DU,,,2022-03-11,,-41.0


In [15]:
df_calc2 = df_calc.groupby(['item', 'order_no'])[['qty']].agg(sum) # 아이템과 오더번호 기준 sum
df_calc2

qty
item   order_no        
FQMADU 21FAA013Q 1  0.0
       21FAA417Q 1  0.0
       21FAA498Q 1  0.0
       21FAA622Q 1  0.0
       21FAA951Q 1  0.0
...                 ...
       22NXC001Q 1  0.0
       22NXC001Q 2  0.0
       22NXC026Q 1  0.0
       22NXC026Q 2  0.0
       22NXC026Q-1  0.0

[1331 rows x 1 columns]

## 문제의 order_no

In [16]:
df_result = df_calc2[df_calc2['qty'] != 0] # 0인 오더를 걷어낸다.
df_result

qty
item   order_no        
FQMADU 22NAA004Q-1  2.0
       22NAA951Q-1  2.0
       22NHA077Q-1  4.0
       22NPC032Q 2 -0.5
       22NRC070Q-1  2.0

### 세부내역

In [17]:
df_result = df_result.reset_index()
order = df_result['order_no'].to_list()
df_temp1 = pd.DataFrame()

for no in order:
    df_temp1 = pd.concat([df_temp1, df_ta[df_ta['order_no'] == no]])

df_temp1 = df_temp1.sort_values(['cdate']).reset_index(drop=True)

df_temp1

,order_no,st,sch_name,item,paitem,sojae,millon,cdate,fact,qty
0,22NAA004Q-1,00,광명중,FQMADU,DU,UGD0407,21FGS030526,2021-06-07,VTF09,2.0
1,22NAA951Q-1,00,옥빛중,FQMADU,DU,UGD0044,22FGS030008,2021-08-10,VPF39,2.0
2,22NHA077Q-1,00,신창중,FQMADU,DU,USX0038,21FGS030166,2021-08-25,VTF09,4.0
3,22NRC070Q-1,00,경남정보고남,FQMADU,DU,UGD0407,22FGS030011,2021-10-07,VPF39,2.0
4,22NPC032Q 2,00,동인고남,FQMADU,DU,UGD0818,20FSC130077,2021-11-16,VPF39,18.0
5,22NPC032Q 2,00,동인고남,FQMADU,DU,UGD0818,21FGS030292,2021-11-16,VPF39,19.5
6,22NPC032Q 2,00,동인고남,FQMADU,DU,UGD0818,20FSC130077,2021-11-29,VPF39,-18.0
7,22NPC032Q 2,00,동인고남,FQMADU,DU,UGD0818,21FGS030292,2021-11-29,VPF39,-20.0


In [18]:
# 혹시나 사용량도 확인해본다.
df_temp2 = pd.DataFrame()
for no in order:
    df_temp2 = pd.concat([df_temp2, df_sa[df_sa['order_no'] == no]])
df_temp2

,order_no,st,sch_name,item,paitem,sojae,millon,cdate,fact,qty


#### 조건부서식 (셀 스타일 배경색 추가, 텍스트 굵게 적용)

In [19]:
def draw_color_in_bursojae(value, color, font_weight):
    if str(value) == '00':
        fill1 = color
        fill2 = font_weight
    else:
        fill1 = "inherit"
        fill2 = "inherit"

    return f"background-color: {fill1};font-weight: {fill2}"

df_temp1 = df_temp1.style.applymap(draw_color_in_bursojae, color='#FFC000', font_weight='bold')
df_temp1

,order_no,st,sch_name,item,paitem,sojae,millon,cdate,fact,qty
0,22NAA004Q-1,00,광명중,FQMADU,DU,UGD0407,21FGS030526,2021-06-07 00:00:00,VTF09,2.000000
1,22NAA951Q-1,00,옥빛중,FQMADU,DU,UGD0044,22FGS030008,2021-08-10 00:00:00,VPF39,2.000000
2,22NHA077Q-1,00,신창중,FQMADU,DU,USX0038,21FGS030166,2021-08-25 00:00:00,VTF09,4.000000
3,22NRC070Q-1,00,경남정보고남,FQMADU,DU,UGD0407,22FGS030011,2021-10-07 00:00:00,VPF39,2.000000
4,22NPC032Q 2,00,동인고남,FQMADU,DU,UGD0818,20FSC130077,2021-11-16 00:00:00,VPF39,18.000000
5,22NPC032Q 2,00,동인고남,FQMADU,DU,UGD0818,21FGS030292,2021-11-16 00:00:00,VPF39,19.500000
6,22NPC032Q 2,00,동인고남,FQMADU,DU,UGD0818,20FSC130077,2021-11-29 00:00:00,VPF39,-18.000000
7,22NPC032Q 2,00,동인고남,FQMADU,DU,UGD0818,21FGS030292,2021-11-29 00:00:00,VPF39,-20.000000


#### 엑셀저장

In [20]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter(f'{start_month}_{end_month}_제조보고서_원단내역.xlsx', engine='xlsxwriter')

# Write each dataframe to a different worksheet.
df_temp1.to_excel(writer, sheet_name='세부내역', index=False)

# Close the Pandas Excel writer and output the Excel file.
writer.save()